# Analysis of Flight Delays in the United States (DTSA 5304 Final Project)

**2023-12-04**

## Download Links

You can download the reproducible Notebook from which this knitted HTML was generated from the links below.

- [DTSA5304_Analysis_of_Flight_Delays_in_US.ipynb](https://github.com/estersstirrer/DTSA5304/blob/main/DTSA5304_Analysis_of_Flight_Delays_in_US.ipynb) ([Raw](https://raw.githubusercontent.com/estersstirrer/DTSA5304/main/DTSA5304_Analysis_of_Flight_Delays_in_US.ipynb))

If your browser does not handle the above links well, please visit [my github.com repository](https://github.com/estersstirrer/DTSA5304/){target="_blank"} to find and grab [the Notebook](https://github.com/estersstirrer/DTSA5304/blob/main/DTSA5304_Analysis_of_Flight_Delays_in_US.ipynb) from there.



## Abstract

This report conducts a comprehensive analysis of flight delays within the United States, utilizing the Flight Delay and Cancellation Dataset spanning from 2019 to 2023. The main goal is to pinpoint airlines and routes that are most frequently delayed. By integrating Pandas for data management and Altair for data visualization, this research offers an in-depth examination of delay trends during both departure and arrival stages across different airlines and airports. The study further explores the ramifications of these delays on various flight routes. The findings of this analysis aim to provide travelers with valuable insights for making informed decisions and assist airlines in enhancing their time management strategies.

## Introduction

This report examines [the Flight Delay and Cancellation Dataset (2019-2023)](https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023) covering the period from 2019 to 2023. Details of this dataset will be discussed in subsequent sections. The primary focus is to identify U.S. airlines that frequently encounter delays, determining which companies experience the most significant delays and exploring the potential causes of these delays, assuming the dataset provides sufficient information.

The analysis also includes an examination of various flight routes to identify those more prone to delays. Factors such as weather conditions, airspace congestion, or other operational issues will be considered in understanding the reasons behind these delays.

Furthermore, the report aims to integrate these findings by visualizing the correlation between specific airlines and the flight routes they operate, highlighting the most delay-prone combinations. This visualization will not only help travelers in making better flight choices but also guide airlines in pinpointing areas requiring improvement in punctuality.

## Flight Delay and Cancellation Dataset (2019-2023)

### Dataset Overview

[The Flight Delay and Cancellation Dataset (2019-2023)](https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023) comprises extensive information compiled by the U.S. Department of Transportation. It details flight delays and cancellations in the U.S. from January 2019 to August 2023, forming part of a larger project tracking flight times since 1987. This dataset is publicly available on the Department of Transportation's website.

### Dataset Composition

The dataset encompasses comprehensive information, including flight origins and destinations, duration of delays and cancellations, and their timings in minutes and local times. While providing explanations for these delays and cancellations, the dataset's level of detail in these explanations may vary. This data aids in a broader understanding of flight punctuality and the challenges airlines face in maintaining timely operations.

### Data Compilation Methodology

This dataset represents a concerted effort in data compilation, updated until August 2023 and finalized in November 2023. The data is organized monthly and aggregated annually. Tools such as csvkit, Python, and Excel were employed to structure, modify, and select key data points. This dataset builds upon a predecessor that covered flight delays and cancellations from 2009 to 2018, maintaining consistency in format while incorporating more recent data.

## Library Dependencies

In this data analysis project, we are using three important tools:

1. **Pandas**: This is a tool we use for working with data. It helps us organize, clean, and make sense of our data. Pandas makes it easy to deal with big sets of data and do things like sort them, group them together, or change them as needed.

2. **Altair**: We use Altair for creating charts and graphs. It's a simple but powerful tool that lets us turn our data into interactive and easy-to-understand visualizations. Altair works well with Pandas, which means we can easily go from organizing our data to showing it in a visual way.

3. **Camel-Converter**: This tool is a lesser-known but really handy for making sure all our column names looks the same, especially when we get it from open repositories. It changes the format of the names in our data so that they all follow the same style, like all snake_case or all PascalCase. This keeps our data neat and consistent.

4. **Kaggle**: To access and download the dataset directly from the Kaggle repository, we use the Kaggle Python library/CLI tool. This streamlines the process of obtaining the dataset for our analysis, allowing for direct integration into our workflow.

Each of these tools has a special role in our project. They help us manage, understand, and show our data in a clear and effective way.


In [ ]:
%pip install -q pandas altair camel-converter kaggle

import pandas as pd
import altair as alt
from camel_converter import to_pascal

pd.set_option("display.max_columns", None)

## Data Preparation

In this section, I outline the steps taken to prepare the data for analysis. The process involved downloading, consolidating, cleaning, pruning, casting data types, and sorting. This was crucial for ensuring the data was in a suitable format for detailed analysis.

### Downloading Dataset

This process involved two steps: downloading and unzipping.

1. **Downloading**:
I started by fetching the data from the online source, [kaggle.com](https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023). This data was compressed into a ZIP file for easy transfer.

2. **Unzipping**:
After downloading, I used the `zipfile` module of Python to `unzip` the file. This meant unpacking the contents into a folder, making the data accessible and ready for analysis.

This preparation was crucial to begin the detailed data analysis.


In [ ]:
import os
import zipfile

zip_path = './flight-delay-and-cancellation-dataset-2019-2023.zip'
data_dir = './data'

if not os.path.exists(zip_path):
    pass
    !KAGGLE_CONFIG_DIR=`pwd` kaggle datasets download -d patrickzel/flight-delay-and-cancellation-dataset-2019-2023
    !echo 'Downloading ... done!'
else:
    pass
    !echo f"The archive file {zip_path} exist. Skip downloading."

if not os.path.exists(data_dir):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip:
            zip.extractall(dest_dir)
        print("ZIP file extracted successfully.")
    except zipfile.error as e:
        print(f"Error: The file is not a zip file or it is corrupted: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    pass
    !echo f"The data directory {data_dir} exist. Skip unzipping."


### Loading the Airline Code Dictionary

Before analyzing the flight delay dataset, we first load auxiliary data, the airline code dictionary, from an accompanying CSV file into a Python dictionary using the Pandas library.

First, the CSV file is read into a DataFrame, a tabular structure in Pandas. The 'Code' column from the CSV is set as the key, and the 'Description' column as the value.

This DataFrame is converted into a dictionary with key-value pairs corresponding to these columns. The final output is a dictionary representing the CSV data, which facilitates easy data access and manipulation.

In [ ]:
# Load the Airline Code CSV file into a DataFrame
df_airline_code = pd.read_csv(f"{data_dir}/AIRLINE_CODE_DICTIONARY.csv")

# Convert the DataFrame into a dictionary
# 'Code' is used as the key, and 'Description' as the value
airline_codes = df_airline_code.set_index('Code')['Description'].to_dict()

### Data Consolidation

First, I gathered data from multiple CSV files, each representing a different year from 2019 to 2023.

Using a loop, I read each file and appended its contents to a list. These individual dataframes were then combined into a single dataframe `df_raw` using the concat method.

This created a comprehensive dataset encompassing all years.

In [ ]:
df_all = []
for y in range(2019, 2024):
    df_all.append(pd.read_csv(f"{data_dir}/{y}.csv"))

df_raw = pd.concat(df_all, axis=0)
for df in df_all:
    del df
del df_all
df_raw.head()

### Cleaning Headers

To standardize column names, I used a custom function `clean_headers`. This function removed non-ASCII and non-printable characters, replaced spaces with underscores, and converted names to PascalCase. Applying this function, I transformed the column headers of `df_raw`, resulting in a cleaner and more consistent structure in `df_renamed`.

In [ ]:
def clean_headers(name):
    if isinstance(name, str):
        name = "".join(c if (c.isascii() & c.isprintable()) else " " for c in name)
        name = (
            "".join(c if c.isalnum() else " " for c in name).strip().replace(" ", "_")
        )
    return to_pascal(name)


df_renamed = df_raw.rename(columns=clean_headers)
df_renamed.head()

### Pruning Columns

The next step was to prune unnecessary columns to focus on relevant data. I selectively kept columns crucial for the analysis, such as flight dates, airline codes, origin and destination airports, times, delays, and flight distance. This pruning resulted in a more manageable dataset `df_pruned`.

In [ ]:
df_pruned = df_renamed.loc[
    :,
    [
        "FlDate",
        "AirlineCode",
        "DotCode",
        # "FlNumber",
        "Origin",
        # "OriginCity",
        "Dest",
        # "DestCity",
        "CrsDepTime",
        # "DepTime",
        "DepDelay",
        # "TaxiOut",
        # "WheelsOff",
        # "WheelsOn",
        # "TaxiIn",
        "CrsArrTime",
        # "ArrTime",
        "ArrDelay",
        "Cancelled",
        # "CancellationCode",
        "Diverted",
        # "CrsElapsedTime",
        # "ElapsedTime",
        # "AirTime",
        # "Distance",
        # "DelayDueCarrier",
        # "DelayDueWeather",
        # "DelayDueNas",
        # "DelayDueSecurity",
        # "DelayDueLateAircraft",
        # "FlYear",
        # "FlMonth",
        # "FlDay",
    ],
]

### Casting Data Types

To optimize memory usage and ensure correct data types for analysis, I cast several columns to more appropriate data types using the `cast_types` function. For instance, dates were converted to datetime objects, and some numeric fields were cast to integer types. The `bool` type was applied to binary columns like 'Cancelled' and 'Diverted'. The resulting dataframe `df_typed` was not only more memory-efficient but also more suitable for analysis.

In [ ]:
def cast_types(df):
    df["FlDate"] = pd.to_datetime(df["FlDate"])
    df["DotCode"] = df["DotCode"].astype("int16")
    df["CrsDepTime"] = df["CrsDepTime"].astype("int16")
    df["CrsArrTime"] = df["CrsArrTime"].astype("int16")
    df["Cancelled"] = df["Cancelled"].astype("bool")
    df["Diverted"] = df["Diverted"].astype("bool")
    return df


df_typed = cast_types(df_pruned)
df_typed.info()
df_typed.describe(include="all")

### Sorting the Data

Finally, I sorted df_typed by flight date (FlDate) to ensure the data was in chronological order. This step is essential for any time series analysis and also helps in visualizing trends over time.

In [ ]:
df_sorted = df_typed.sort_values(by=['FlDate'])
df_sorted.head()

## Analyzing Data with Visualization

### Comparative Analysis of Mean Arrival and Departure Delays by Airline

First, we will compare the average delay times for take-off (departure) and landing (arrival) across different airlines, using two separate bar charts to visualize each type of delay.

- **Take-off Delays (Departure)**: The first chart shows how long, on average, each airline is delayed when taking off. The Y-axis shows the delay time in minutes, and the X-axis lists the airline names. This chart is colored salmon. It helps us see which airlines usually have longer wait times before they take off.

- **Landing Delays (Arrival)**: The second chart is about how late airlines are when they land. It's set up just like the first chart. The delay time is shown on the Y axis, and the airlines are listed on the X axis. This chart is steel blue, which makes it easy to tell apart from the take-off delay chart. It shows us which airlines tend to be late when they arrive.

Both charts are organized to show the airlines with the most delays at the top, making it easier to compare. When you look at them side by side, you get a good idea of which airlines are often late both when they take off and when they land. By using the same scale for the delay times on both charts, we can compare them fairly. This way, we get a clear picture of how well airlines keep to their schedules from start to finish.

In [ ]:
alt.data_transformers.disable_max_rows()

df = df_sorted

In [ ]:
# Aggregation for departure delays
df_departure_agg = (
    df.groupby(by=["AirlineCode"], as_index=False)
    .mean("DepDelay")[["AirlineCode", "DepDelay"]]
    .sort_values(["DepDelay"], ascending=[False])
)
df_departure_agg["Airline"] = df_departure_agg["AirlineCode"].map(airline_codes)
dep_delay_chart = (
    alt.Chart(df_departure_agg)
    .mark_bar()
    .encode(
        y=alt.Y("DepDelay", title="Mean Departure Delay (minutes)"),
        x=alt.X("Airline", title="Airline Name", sort=None),
        color=alt.value("salmon"),
    )
    .properties(title="Mean Departure Delay by Airline")
)

# Aggregation for arrival delays
df_arrival_agg = (
    df.groupby(by=["AirlineCode"], as_index=False)
    .mean("ArrDelay")[["AirlineCode", "ArrDelay"]]
    .sort_values(["ArrDelay"], ascending=[False])
)
df_arrival_agg["Airline"] = df_arrival_agg["AirlineCode"].map(airline_codes)
arr_delay_chart = (
    alt.Chart(df_arrival_agg)
    .mark_bar()
    .encode(
        y=alt.Y("ArrDelay", title="Mean Arrival Delay (minutes)"),
        x=alt.X("Airline", title="Airline Name", sort=None),
        color=alt.value("steelblue"),
    )
    .properties(title="Mean Arrival Delay by Airline")
)

# Concatenating charts
combined_chart = alt.hconcat(dep_delay_chart, arr_delay_chart).resolve_scale(y="shared")

combined_chart.display()

### Analysis of Airport-Specific Flight Delays: Best and Worst 10 Airports

In this section, we explore the average departure and arrival delays at different airports. The analysis focuses on identifying the top and worst 10 airports in terms of delay times. We use two separate bar charts to present this data clearly.

- **Departure Delays by Airport**: The first chart shows the mean departure delay times at origin airports. We aggregated the data by airport and calculated the average delay for each. The chart highlights the 10 airports with the longest and shortest average departure delays, giving us insights into which airports are frequently facing delays and which are most efficient. The delays are represented in salmon color for easy visualization.

- **Arrival Delays by Airport**: The second chart follows a similar approach but focuses on the destination airports. It illustrates the mean arrival delay times, again showcasing the top and worst 10 airports. This helps us understand which airports are efficient in handling incoming flights and which ones tend to have longer waiting times upon arrival. The chart uses a steel blue color for distinction.

Both charts are aligned side by side for a comparative view, with a shared scale for delay times on the Y-axis. This setup enables a comprehensive analysis of airport performance regarding both departure and arrival delays, offering a complete picture of where travelers might experience the most significant delays.

In [ ]:
def head_n_tail(df, field, n):
    top_n = df.nlargest(n, field)
    bottom_n = df.nsmallest(n, field)
    return pd.concat([top_n, bottom_n])

# Aggregation for departure delays
df_departure_agg = (
    df.groupby(by=["Origin"], as_index=False)
    .mean("DepDelay")[["Origin", "DepDelay"]]
    .sort_values(["DepDelay"], ascending=[True])
)
# df_departure_agg["OriginAirport"] = df_departure_agg["Origin"].map(airport_codes)
df_departure_agg_trimmed = head_n_tail(df_departure_agg, 'DepDelay', 10)

dep_delay_chart = (
    alt.Chart(df_departure_agg_trimmed)
    .mark_bar()
    .encode(
        y=alt.Y("DepDelay", title="Mean Departure Delay (minutes)"),
        x=alt.X("Origin", title="Origin Airport", sort=None),
        color=alt.value("salmon"),
    )
    .properties(title="Best and Worst 10 Airports for Mean Departure Delays")
)

# Aggregation for arrival delays
df_arrival_agg = (
    df.groupby(by=["Dest"], as_index=False)
    .mean("ArrDelay")[["Dest", "ArrDelay"]]
    .sort_values(["ArrDelay"], ascending=[True])
)
# df_arrival_agg["DestAirport"] = df_arrival_agg["Dest"].map(airport_codes)
df_arrival_agg_trimmed = head_n_tail(df_arrival_agg, 'ArrDelay', 10)
arr_delay_chart = (
    alt.Chart(df_arrival_agg_trimmed)
    .mark_bar()
    .encode(
        y=alt.Y("ArrDelay", title="Mean Arrival Delay (minutes)"),
        x=alt.X("Dest", title="Destination Airport", sort=None),
        color=alt.value("steelblue"),
    )
    .properties(title="Best and Worst 10 Airports for Mean Arrival Delays")
)

# Concatenating charts
combined_chart = alt.hconcat(dep_delay_chart, arr_delay_chart).resolve_scale(y="shared")

combined_chart.display()

### Analysis of Flight Route Delays: Comparing the Best and Worst

Let's dive a bit deeper into our analysis. We're now exploring how late flights usually are when traveling between different airports. Our goal was to identify flight paths with the most and least delays. First, we reviewed all the flights, paying special attention to their starting points and destinations. From there, we calculated the average arrival delay for each flight path.

Next, we created a list that displayed each flight path, like "AirportA to AirportB". We then selected the 30 flight paths with the longest delays and the 30 with the shortest delays. These were combined to identify which paths were the best and worst in terms of punctuality.

The results were presented in a bar chart, neatly divided into two sections. One section illustrates the delay durations in minutes along the vertical axis, while the other plots the specific flight paths horizontally. This layout makes it straightforward to identify which flights are generally delayed and which tend to arrive on time. It effectively illustrates the variability in delay times across different flight routes, offering a clear perspective on airline punctuality for various routes.

In [ ]:
df_agg = df.groupby(by=["Origin", "Dest"], as_index=False).mean("ArrDelay")[
    ["Origin", "Dest", "ArrDelay"]
]
df_agg["OriginDest"] = (
    df_agg["Origin"].astype(str) + " to " + df_agg["Dest"].astype(str)
)
df_agg_top_bottom = head_n_tail(df_agg, "ArrDelay", 30)

selection = alt.selection_point(fields=[])
chart = (
    alt.Chart(df_agg_top_bottom)
    .mark_bar()
    .encode(
        y=alt.Y("ArrDelay", title="Mean Arrival Delay (minutes)"),
        x=alt.X("OriginDest", title="Flight Route (Origin Airport to Destination Airport)", sort=None),
    )
    .properties(title="Best and Worst 30 Flight Routes for Mean Arrival Delays")
    .add_params(selection)
)
chart.display()

### Interlude - the Complexity of Visualizing Airline Delays

We're embarking on an intriguing challenge: depicting airlines' punctuality clearly and clearly.

An interesting suggestion emerged after sharing earlier visualizations with my colleagues for feedback. Some pointed out the value of visualizing the average arrival delays for airlines across all their flight routes. This comprehensive approach would make it easier to determine which airline might be the best choice for a specific flight route.

Our goal is now twofold. Firstly, we aim to illustrate the delays for a single flight path, which is relatively straightforward. However, the real test comes when expanding this analysis to encompass all flight paths of every airline. This is not easy, considering the sheer number of routes and airlines. The main hurdle is to avoid a cluttered and confusing representation.

We need to devise a visualization method that successfully integrates this vast array of data into a format that's both informative and user-friendly. The key lies in achieving a balance – presenting a comprehensive overview of airline punctuality without overwhelming the viewer.

### Interactive Visualization of Airline Delay Times Across Flight Routes

In our final analysis, we visualized the mean arrival delay times of airlines for each flight route. We began by grouping our flight data by origin airport, destination airport, and airline code, calculating the average delay time for these groups. To clarify, we created the 'OriginToDest' field, combining each flight's origin and destination, like 'LAX to LAS.'

Enhancing user engagement, we incorporated a dropdown menu in our chart, enabling users to select specific flight routes. This interactive element filters the data to showcase the average delay times for airlines on the route chosen. The resultant bar chart vividly presents the mean arrival delay for each airline on a preferred route, offering a comprehensive comparison of airline punctuality across various routes. The "Mean Delay Time by Airline per Flight Route" chart includes a subtitle encouraging users to select a route for detailed insights. This visualization is designed to offer an in-depth perspective on airline performance regarding arrival punctuality, tailored to the user's specific interests.

In [ ]:
df_agg = (
    df.groupby(by=["Origin", "Dest", "AirlineCode"], as_index=False)
    .mean("ArrDelay")[["Origin", "Dest", "AirlineCode", "ArrDelay"]]
    .sort_values(["ArrDelay"], ascending=[True])
)
df_agg["OriginToDest"] = (
    df_agg["Origin"].astype(str) + " to " + df_agg["Dest"].astype(str)
)
df_agg["AirlineName"] = df_agg["AirlineCode"].map(airline_codes)

criteria = sorted(df_agg["OriginToDest"].unique())
dropdown = alt.binding_select(options=criteria)
selection = alt.selection_point(
    fields=["OriginToDest"],
    bind=dropdown,
    name="Select",
    value=[{"OriginToDest": 'LAX to LAS'}],
)

scatter_plot = (
    alt.Chart(df_agg)
    .mark_bar()
    .encode(
        x=alt.X("AirlineName", title="Airline from Selected Airport", sort=None),
        y=alt.Y("ArrDelay", title="Mean Arrival Delay (minute)", sort=None),
        tooltip=["ArrDelay"],
    )
    .add_params(selection)
    .transform_filter(selection)
    .properties(
        title={
            "text": "Mean Delay Time by Airline per Flight Route",
            "subtitle": [
                "Select a route to show the average delay time by airline for the selected flight route.",
            ],
            # "color": "black",
            # "font": "Courier",  # Font of the title
            # "fontSize": 20,  # Font size
            "anchor": "start",  # Alignment (start, middle, end)
            "subtitleColor": "gray",  # Subtitle color
        }
    )
)

scatter_plot.display()

## Evaluation

To gauge the efficacy of our Flight Delay and Cancellation Visualization, our focus will be on several key aspects:

### Participant Recruitment

For this evaluation, I recruited both my colleagues and family members, given that the visualization is designed specifically for travelers undertaking business and leasure trips.

### Measurement Criteria

Evaluating the visualization's impact requires translating user feedback into quantifiable numerical scores. This can be achieved by having participants respond to questions using a fixed scale, such as a range from 1 to 10.

The following questions were posed to the users:

- **Question 1**: Did the visualization effectively aid in determining your Flight Route for the trip?
- **Question 2**: Did the visualization effectively aid in selecting the Airline Company for your trip?
- **Question 3**: Was the visualization user-friendly in terms of finding the required information?

### Assess the Feedback

We analyzed participant scores and comments to identify strengths and areas for improvement in the visualization. This analysis helped refine the visualization's designs, enhancing its usefulness in travel planning. The evaluation guided improvements in performance and user experience.

## Conclusion

The analysis of the Flight Delay and Cancellation Dataset (2019-2023) has provided valuable insights into the patterns of flight delays in the U.S. airline industry. Key findings include identifying airlines and flight routes with the highest average delays. The study revealed that some airlines consistently experience longer delays at the departure and arrival stages. Additionally, certain airports and flight routes were identified as more prone to delays, which can be attributed to airport traffic and operational challenges.

The visualizations created using Altair facilitated an in-depth understanding of these delay patterns, highlighting the complexity of factors contributing to airline punctuality. The interactive visualization, in particular, offered a user-friendly approach to exploring delay times across different flight routes for various airlines. This feature can be a practical tool for travelers like me and my colleagues to assess and improve flight punctuality before business trips.

Overall, this study emphasizes the multifaceted nature of flight delays and the importance of continuous monitoring and analysis to enhance the efficiency of air travel. The insights gained can help passengers in making better-informed travel choices. Future research could further explore the underlying causes of delays and extend the analysis to include additional variables such as weather patterns, seasonal variations, and airline-specific operational strategies.